# Algoritmia
## Práctica 2
El objetivo de esta práctica es trabajar con iteradores y generadores.

Se pide la implementación de las funciones que aparecen a continuación. 

En el cuerpo de cada función hay una instrucción ```pass```, se debe sustituir por la implementación adecuada.

Para cada función que se pide se proporcionan algunos tests. Las implementaciones deberían superar estos tests.

## Preámbulo

In [ ]:
# Importaciones
import unittest
from itertools import chain, count, cycle, repeat, zip_longest

In [ ]:
# Número de iteraciones que se usan en algunos tests.
# Cuando se está realizando la práctica puede ser conveniente utilizar un valor más pequeño para que vaya más rápido.
# Pero una vez finalizada debería poder ejecutarse con este valor en un tiempo de minutos.
num_iteraciones_test = 10**7

## Secuencia generalizada de Fibonacci
En la secuencia de Fibonacci, cada valor se obtiene sumando los dos anteriores. Se considera una generalización en la que cada valor se obtiene sumando los *k* anteriores:
- F(0) = ... = F(k-1) = 1
- F(n) = F(n-1) + ... + F(n-k+1)

### Función `fibonacci_generalizado`

In [ ]:
def fibonacci_generalizado(k, iniciales = None):
    """
    Genera indefinidamente valores de la secuencia generalizada de Fibonacci.
    Cada valor, salvo los iniciales, es la suma de los k anteriores.
    Los valores iniciales, que deben ser k, son los valores de F(0) ... F(k-1).
    El valor por defecto de los valores iniciales es 1.
    El espacio de memoria utilizado debería ser O(k)
    """
    if iniciales is None:
        iniciales = [1] * k
    assert len(iniciales) == k
    yield from iniciales
    anteriores = iniciales
    while True:
        yield sum(anteriores)
        anteriores = anteriores[1:] + [sum(anteriores)]

### Tests para `fibonacci_generalizado`

In [ ]:
class TestFibonacciGeneralizado(unittest.TestCase):
    
    def setUp(self):
        # Distintos casos de prueba formados por k (el orden de la secuencia),
        # los valores iniciales y el inicio de la secuencia que se debe
        # generar
        self.casos_prueba = (
            (2, None, [1, 1, 2, 3, 5, 8, 13, 21, 34]),
            (2, [2, 2], [2, 2, 4, 6, 10, 16, 26, 42]),
            (3, None, [1, 1, 1, 3, 5, 9, 17, 31, 57]),
            (4, None, [1, 1, 1, 1, 4, 7, 13, 25, 49]),
        )
        
    def test_1(self):
        # Primera comprobación de los casos de prueba
        for k, iniciales, secuencia in self.casos_prueba:
            i = 0
            for v in fibonacci_generalizado(k, iniciales):
                if i >= len(secuencia):
                    break
                self.assertEqual(v, secuencia[i])
                i += 1
            assert i == len(secuencia)

    def test_2(self):
        # Segunda comprobación de los casos de prueba
        for k, iniciales, secuencia in self.casos_prueba:
            for v, w in zip(fibonacci_generalizado(k, iniciales), secuencia):
                self.assertEqual(v, w)
                
    def test_3(self):
        # Tercera comprobación de los casos de prueba
        for k, iniciales, secuencia in self.casos_prueba: 
            generador = fibonacci_generalizado(k, iniciales)
            for v in secuencia:
                self.assertEqual(v, next(generador))
                
    def test_muchos_valores(self):
        # Comprobación de que se generan muchos valores
        generador = fibonacci_generalizado(4, [0, 0, 0, 0])
        for _ in range(num_iteraciones_test):
            self.assertEqual(next(generador), 0)

## Iterador repetido
Dado un iterador o iterable, se quiere generar sus elementos repetidamente, donde el número de repeticiones de cada elemento viene dado por un segundo iterador o iterable.

### Función `iter_repetido`

In [ ]:
def iter_repetido(itera, repeticiones):
    """
    Genera los elementos del primer argumento tantas veces como el elemento 
    correspondiente del segundo argumento.
    Se espera que los elementos del segundo argumento sean números naturales.
    El primer elemento del primer argumento se genera tantas veces como el 
    primer elemento del segundo argumento, ... el elemento i-ésimo del primer 
    argumento se genera tantas veces como el elemento i-ésimo del segundo
    argumento...
    Si el número de elementos de los dos argumentos fuera diferente, se
    generarán elementos hasta que uno se quede sin elementos.
    """

    for iter, repe in zip(itera, repeticiones):
        yield from repeat(iter, repe)

### Tests para `iter_repetido`

In [ ]:
class TestIterRepetido(unittest.TestCase):
    
    def setUp(self):
        # Distintos casos de prueba formados por el iterable, las repeticiones
        # y la lista de valores que se deben generar
        self.casos_prueba = (
            ("abc", [3, 0, 2], ['a', 'a', 'a', 'c', 'c']),
            ("abcd", [3, 0, 2], ['a', 'a', 'a', 'c', 'c']),
            ("abc", [3, 0, 2, 4], ['a', 'a', 'a', 'c', 'c']),
            (range(3), range(1, 4), [0, 1, 1, 2, 2, 2])
    )
        
    def test_1(self):
        # Primera comprobación de los casos de prueba
        for iterable, repeticiones, lista in self.casos_prueba:
            self.assertEqual(list(iter_repetido(iterable, repeticiones)), 
                             lista)

    def test_2(self):
        # Segunda comprobación de los casos de prueba
        for iterable, repeticiones, lista in self.casos_prueba:
            for v, w in zip_longest(iter_repetido(iterable, repeticiones), 
                                    lista):
                self.assertEqual(v, w)

    def test_3(self):
        # Tercera comprobación de los casos de prueba        
        for iterable, repeticiones, lista in self.casos_prueba:
            generador = iter_repetido(iterable, repeticiones)
            for v in lista:
                self.assertEqual(v, next(generador))
                
    def test_muchos_valores_1(self):
        # Primera comprobación de que se generan muchos valores
        generador = iter_repetido(count(), repeat(1))
        for i in range(num_iteraciones_test):
            self.assertEqual(i, next(generador))

    def test_muchos_valores_2(self):
        # Segunda comprobación de que se generan muchos valores
        repeticiones = [3, 0, 1]
        generador = iter_repetido(count(), cycle(repeticiones))
        r = 0
        for i in range(num_iteraciones_test):
            for _ in range(repeticiones[r]):
                self.assertEqual(i, next(generador))
            r = r + 1 
            if r == len(repeticiones): r = 0            

## Mezcla de iteradores ordenados
Dados dos iteradores o iterables que generan valores ordenados, se quiere mezclar los elementos de ambos. La mezcla consiste en generar en orden los elementos de los dos iteradores.

### Función `iter_mezcla`

In [ ]:
def iter_mezcla(iter_1, iter_2):
    """
    Dados dos iteradores o iterables, suponiendo que ambos generan valores en
    orden, se generan los elementos de ambos de manera ordenada.
    La cantidad de memoria usada debe ser O(1).
    """

    iter_1 = iter(iter_1)
    iter_2 = iter(iter_2)

    try:
        v1 = next(iter_1)
    except StopIteration:
        yield from iter_2
        return

    try:
        v2 = next(iter_2)
    except StopIteration:
        yield v1
        yield from iter_1
        return

    while True:
        if v1 < v2:
            yield v1
            try:
                v1 = next(iter_1)
            except StopIteration:
                yield v2
                yield from iter_2
                return
        else:
            yield v2
            try:
                v2 = next(iter_2)
            except StopIteration:
                yield v1
                yield from iter_1
                return

### Tests para `iter_mezcla`

In [ ]:
class TestIterMezcla(unittest.TestCase):
    
    def setUp(self):
        # Distintos casos de prueba formados por los dos iterables de entrada y
        # un tercero con la salida esperada
        self.casos_prueba = (
            (range(100), range(100, 200), range(200)),
            (range(0, 100, 2), range(1, 100, 2), range(100)),
            (range(0, 100, 4), range(2, 100, 4), range(0, 100, 2)),
            (range(0, 100), range(200, 300), 
             chain(range(0, 100), range(200, 300))),
            (range(100), range(100), (x for x in range(100) for _ in range(2))),
            (range(0, 100, 3), range(0, 100, 5), 
             sorted(chain(range(0, 100, 3), range(0, 100, 5)))),
            (range(num_iteraciones_test), 
             range(num_iteraciones_test, 2*num_iteraciones_test),
             range(2*num_iteraciones_test)),
            (range(0, num_iteraciones_test, 2), 
             range(1, num_iteraciones_test, 2), 
             range(num_iteraciones_test)),
            (range(num_iteraciones_test), range(num_iteraciones_test), 
             (x for x in range(num_iteraciones_test) for _ in range(2)))
        )
        
    def test_1(self):
        # Primera comprobación con los casos de prueba
        for it_1, it_2, it_resultado in self.casos_prueba:
            for v, w in zip_longest(iter_mezcla(it_1, it_2), it_resultado):
                self.assertEqual(v, w) 

    def test_2(self):
        # Segunda comprobación con los casos de prueba
        # La diferencia con la primera es el orden en que se pasan los iterables
        for it_1, it_2, it_resultado in self.casos_prueba:
            for v, w in zip_longest(iter_mezcla(it_2, it_1), it_resultado):
                self.assertEqual(v, w) 


## Ejecución de tests

In [ ]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)